In [1]:
import pandas as pd
import os

In [16]:
folder_path = r"Dataset/"

# excel_file = r"Dataset/CB IG Compiled Coding Set 1 - 2020 (Revised 2.0).xlsx"
filtered_dfs = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    
    if filename.endswith('.xlsx'):
        excel_file = os.path.join(folder_path, filename)
        xlsx = pd.ExcelFile(excel_file)
        sheet_names = xlsx.sheet_names

        dataframe = {}
        defined_column_name = ["Comment_Number",
                               "Commenter_Username",
                               "Comment",
                               "Comment_Post_Time",
                               "Overall_CB_Status",
                               "CB_Type",
                               "Sexual_Type1", "Unnamed_1", "Unnamed_2",
                               "Sexual_Type2", "Unnamed_3", "Unnamed_4",
                               "Physical_Appearance", "Unnamed_5", "Unnamed_6",
                               "Race", "Unnamed_7", "Unnamed_8",
                               "Intellectual", "Unnamed_9", "Unnamed_10",
                               "Religious", "Unnamed_11", "Unnamed_12",
                               "General_Hate", "Unnamed_13", "Unnamed_14",
                               "Purpose_of_CB",
                               "Insult", "Unnamed_15", "Unnamed_16",
                               "Defensive", "Unnamed_17", "Unnamed_18",
                               "Directionality", 
                               "Directed_Username", "Unnamed_19", "Unnamed_20",
                               "Other_Aspects",
                               "Depression", "Unnamed_21", "Unnamed_22",
                               "Suicides", "Unnamed_23", "Unnamed_24",
                               "Stress", "Unnamed_25", "Unnamed_26",
                               "Discrimination", "Unnamed_27", "Unnamed_28"]
        
        for sheet_name in sheet_names:
            dataframe[sheet_name] = pd.read_excel(excel_file, sheet_name)
            try:
                dataframe[sheet_name].columns = defined_column_name
            except:
                print(f'{filename} : {sheet_name}')

        for item in dataframe.values():
            item.columns = defined_column_name
            item = item.drop(item.index[:2])
            columns_to_drop = [col for col in item.columns if any(substring in col for substring in ["Comment_Number",
                                                                                                     "Unnamed",
                                                                                                     "Commenter_Username",
                                                                                                     "Comment_Post_Time",
                                                                                                     "CB_Type", 
                                                                                                     "Purpose_of_CB",
                                                                                                     "Insult",
                                                                                                     "Religious",
                                                                                                     "Defensive",
                                                                                                     "Other_Aspects",
                                                                                                     'Suicides',
                                                                                                     "Stress",
                                                                                                     "Depression",
                                                                                                     "Discrimination",
                                                                                                     "Directionality", 
                                                                                                     "Directed_Username"])]
            filter_df = item.drop(columns=columns_to_drop).fillna(0)
            filtered_dfs.append(filter_df)


# Concatenate all the filtered dataframes into one
df = pd.concat(filtered_dfs, ignore_index=True)

In [17]:
df = df.drop_duplicates(subset='Comment')

In [18]:
x = df.isna()

# Check if True exists in the DataFrame
if (x == True).any().any():
    print("True exists in the DataFrame.")
else:
    print("True does not exist in the DataFrame.")

True does not exist in the DataFrame.


In [19]:
selected_column = ['Overall_CB_Status', 'Sexual_Type1', 'Sexual_Type2', 'Physical_Appearance', 'Race', 'Intellectual','General_Hate']

for column_name in selected_column:  
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce').fillna(0)

df.describe()

,Overall_CB_Status,Sexual_Type1,Sexual_Type2,Physical_Appearance,Race,Intellectual,General_Hate
count,7943.000000,7943.00000,7943.000000,7943.000000,7943.000000,7943.000000,7943.000000
mean,0.134080,0.02883,0.013471,0.019388,0.013723,0.019262,0.068362
std,0.341867,0.16734,0.115287,0.137894,0.116345,0.137454,0.252382
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
# Print rows where one of the attributes is equal to 3
df = df[(df['Overall_CB_Status'] != 3)]
df.describe()

,Overall_CB_Status,Sexual_Type1,Sexual_Type2,Physical_Appearance,Race,Intellectual,General_Hate
count,7942.000000,7942.000000,7942.000000,7942.000000,7942.000000,7942.000000,7942.000000
mean,0.133719,0.028834,0.013473,0.019391,0.013725,0.019265,0.068371
std,0.340372,0.167350,0.115295,0.137902,0.116352,0.137462,0.252397
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
# Drop rows where A = 1, but B, C, and D are all 0
condition = (df['Overall_CB_Status'] == 1) & (df['Sexual_Type1'] == 0) & (df['Sexual_Type2'] == 0) & (df['Physical_Appearance'] == 0) & ((df['Race'] == 0) & (df['Intellectual'] == 0) & (df['General_Hate'] == 0))

# Use boolean indexing to drop rows
df = df[~condition]

# Reset the index if needed
df.reset_index(drop=True, inplace=True)

In [25]:
for column_name in selected_column:
    value_counts = df[column_name].value_counts()
    print(value_counts)

0    6880
1     941
Name: Overall_CB_Status, dtype: int64
0.0    7592
1.0     229
Name: Sexual_Type1, dtype: int64
0    7714
1     107
Name: Sexual_Type2, dtype: int64
0    7667
1     154
Name: Physical_Appearance, dtype: int64
0.0    7712
1.0     109
Name: Race, dtype: int64
0    7668
1     153
Name: Intellectual, dtype: int64
0.0    7278
1.0     543
Name: General_Hate, dtype: int64


In [26]:
file_path = './preprocess_data.csv'
df.to_csv(file_path, index=False)